In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import tree, neural_network, neighbors, discriminant_analysis

from mva_helper import * # my helper functions

%matplotlib inline
plt.rcParams['figure.figsize'] = (4, 4) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Discrimination in a Single Dimension

### 1D Gaussian Discriminant 

In [ ]:
figure = plt.figure()
ax = plt.gca()

a = np.random.normal(loc = -1, scale = 2, size = 10000)
b = np.random.normal(loc = 3, scale = 1, size = 10000)

r = np.min([a.min(), b.min()]), np.max([a.max(), b.max()])

plt.hist([a, b], bins=30, range=r, histtype = 'step', color = ["b", "r"], label = ["Category 1", "Category 2"], alpha = 1.0)
plt.legend(loc = "upper left", frameon=False)

ax.set_xlabel("Gaussian Discriminants")
figure.savefig('gaus_discr.pdf', bbox_inches='tight', pad_inches=0.1)
plt.show()

### ROC Curve

In [ ]:
cut = np.linspace(-6, 6, 500)
eff = np.array([np.mean(a < c) for c in cut])
rej = np.array([np.mean(b > c) for c in cut])

figure = plt.figure()
ax = plt.gca()

plt.plot(eff, rej, c = "k")
ax.set_xlim(0, 1.07)
ax.set_ylim(0, 1.07)

ax.set_xlabel('Blue Efficiency', color = "b")
ax.set_ylabel('Red Rejection', color = "r")

plt.scatter([1], [1], c = "b", s = 200, marker = "*", linewidth = 0)

figure.savefig('gaus_roc.pdf', bbox_inches='tight', pad_inches=0.1)

plt.show()

## Principal Component Analysis and Linear Discriminant Analysis

In [ ]:
from sklearn import decomposition, discriminant_analysis

In [ ]:
X, y = blob_data(X = 1, Y = 1, noise_level = 0.35, correlated = 0.75)
y[X[:,0] > 0] = 1

plot_classifier(X, y)

In [ ]:
pca = decomposition.PCA(n_components = 2)
X_r = pca.fit(X).transform(X)
print('explained variance ratio (first two components): ' + str(pca.explained_variance_ratio_))

plt.figure()
plt.scatter(X_r[:, 0], X_r[:, 1], c = y, alpha=.8, cmap=plt.cm.viridis)
plt.show()

### Create a Series of Blobs, Overlapping in x and y.

In [ ]:
X, y = blob_data(X = 2, Y = 4, noise_level = 0.25, correlated = 0.8)
K = len(set(y))

cm = [plt.cm.viridis(v) for v in np.linspace(0, 1, K)]

In [ ]:
lda = discriminant_analysis.LinearDiscriminantAnalysis(n_components = 2)
lda.fit(X, y)

plot_classifier(X, y, fn = lda.predict, file = "lda_classifier.pdf")

In [ ]:
X_r2 = lda.fit(X, y)
X_r2 = lda.transform(X)

fig = plt.figure()
plt.scatter(X_r2[:, 0], X_r2[:, 1], c=y, s=40, cmap=plt.cm.viridis, alpha = 0.2, linewidth = 0.1)
fig.savefig('lda_transf.pdf', bbox_inches='tight', pad_inches=0.1)
plt.show()

fig = plt.figure()
ax = plt.gca()

ax.hist([X_r2[y == c, 0] for c in range(K)], 
        bins=30, histtype = 'step', color = cm, alpha = 1.0)
fig.savefig('lda_project.pdf', bbox_inches='tight', pad_inches=0.1)

plt.show()

In [ ]:
# Define blob data with 2×2
X, y = blob_data(X = 2, Y = 2, noise_level = 0.25, correlated = 0.8)
K = len(set(y))

cm = [plt.cm.viridis(v) for v in np.linspace(0, 1, K)]

# create the LDA and fit it to data
lda = discriminant_analysis.LinearDiscriminantAnalysis()
lda.fit(X, y)

# plot it, using my function
plot_classifier(X, y, fn = lda.predict, file = "lda_classifier2.pdf")
X_r2 = lda.fit(X, y)
X_r2 = lda.transform(X)

fig = plt.figure()
plt.scatter(X_r2[:, 0], X_r2[:, 1], c=y, s=40, cmap=plt.cm.viridis, alpha = 0.2, linewidth = 0.1)
fig.savefig('lda_transf2.pdf', bbox_inches='tight', pad_inches=0.1)
plt.show()

fig = plt.figure()
ax = plt.gca()

ax.hist([X_r2[y == c, 0] for c in range(K)], 
        bins=30, histtype = 'step', color = cm, alpha = 1.0)
fig.savefig('lda_project2.pdf', bbox_inches='tight', pad_inches=0.1)

plt.show()

In [ ]:
X, y = spiral_data(K = 6, noise_level = 0.2)

lda = discriminant_analysis.LinearDiscriminantAnalysis(n_components = 2)
lda.fit(X, y)

plot_classifier(X[::5,], y[::5], fn = lda.predict, file = "lda_lollypop.pdf", alpha = 0.3)

## In the real world...

You wouldn't write your own classifier (without a _lot_ more practice).

[scikit learn](http://scikit-learn.org/) implements many of the "standard" classification algorithms.  Let's see what these look like.  

BUT: _be careful!_  This is an incredibly powerful tool, and one that is very easy to misuse.  Check (but do not tune) to your test sample, use well-understood input variables.  And understand who/what you're selecting.

## Nearest Neighbors and Overtraining

In [ ]:
X, y = blob_data(noise_level = 0.5, correlated = 0.5, N = 200)
# X, y = spiral_data(K = 7, N = 30, noise_level = 0.3)

In [ ]:
knn = neighbors.KNeighborsClassifier(1, weights="uniform")

knn.fit(X, y)
plot_classifier(X[:,], y[:], fn = knn.predict, alpha = 0.4, h = 0.01, file = "knn_overtrained.pdf")

knn = neighbors.KNeighborsClassifier(30, weights="uniform")

knn.fit(X, y)
plot_classifier(X[:,], y[:], fn = knn.predict, alpha = 0.4, h = 0.01, file = "knn_k30.pdf")

In [ ]:
# X, y = blob_data(X = 2, Y = 2, N = 1000, noise_level = 0.5, correlated = 0.6)
X, y = spiral_data(K = 5, noise_level = 0.4, N = 1000)

bdt = tree.DecisionTreeClassifier(min_samples_leaf = 1)
bdt.fit(X, y)

plot_classifier(X[::,], y[::], fn = bdt.predict, alpha = 0.4, alpha_pt = 0.06, file = "bdt_overtrained.pdf")

bdt = tree.DecisionTreeClassifier(min_samples_leaf = 15)
bdt.fit(X, y)

plot_classifier(X[::,], y[::], fn = bdt.predict, alpha = 0.4, alpha_pt = 0.06, file = "bdt_leaf15.pdf")


import pydotplus
from IPython.display import Image  
dot_data = tree.export_graphviz(bdt, out_file=None, filled=True, rounded=True, special_characters=True) 
graph = pydotplus.graph_from_dot_data(dot_data)  
graph.write_pdf("bdt_graph.pdf") 
Image(graph.create_png())  

In [ ]:
X, y = spiral_data(K = 5, noise_level = 0.3, N = 1000)
# X, y = blob_data(X = 2, Y = 2, N = 1000, noise_level = 0.5, correlated = 0.6)

mlp = neural_network.MLPClassifier(solver='lbfgs', alpha = 10, random_state=1, hidden_layer_sizes=(100))
mlp.fit(X, y)

plot_classifier(X[:,], y[:], fn = mlp.predict, alpha = 0.4, alpha_pt = 0.05, file = "mlp_classifier.pdf")

## Test Samples and Overtraining

In [ ]:
Xtr, ytr = spiral_data(K = 5, noise_level = 0.6, N = 50)
Xte, yte = spiral_data(K = 5, noise_level = 0.6, N = 1000)

In [ ]:
alpha = [1e-4, 1e-3, 0.05, 0.1, 0.2, 0.4, 0.6, 0.8, 1, 2, 3, 4, 5]
tr_acc, te_acc = [], []
for a in alpha:
    mlp = neural_network.MLPClassifier(solver='lbfgs', alpha = a, random_state=1, hidden_layer_sizes=(100))
    mlp.fit(Xtr, ytr)
    
    tr_acc.append(np.mean(mlp.predict(Xtr) == ytr))
    te_acc.append(np.mean(mlp.predict(Xte) == yte))

In [ ]:
figure = plt.figure()
ax = plt.gca()

plt.semilogx(alpha, tr_acc, c = "g", label = "Training")
plt.semilogx(alpha, te_acc, c = "r", label = "Test")
# ax.set_xlim(0, 1.07)
ax.set_ylim(0.6, 0.86)

ax.set_xlabel('Regularization')
ax.set_ylabel('Accuracy')

plt.legend(frameon=False)

figure.savefig('regularization.pdf', bbox_inches='tight', pad_inches=0.1)

plt.show()

In [ ]:
Xte, yte = spiral_data(K = 5, noise_level = 0.6, N = 5000)

Ntrain = np.array([4, 6, 8, 10, 20, 30, 40, 50, 60, 80, 90, 100, 120, 150, 200, 300, 400, 500, 1000, 2000, 3000, 4000])
tr_acc, te_acc = [], []

for N in Ntrain:

    Xtr, ytr = spiral_data(K = 5, noise_level = 0.6, N = N)

    mlp = neural_network.MLPClassifier(solver='lbfgs', alpha = 0.1, random_state=1, hidden_layer_sizes=(100))
    mlp.fit(Xtr, ytr)
    
    tr_acc.append(np.mean(mlp.predict(Xtr[:,:]) == ytr[:]))
    te_acc.append(np.mean(mlp.predict(Xte) == yte))
    print(N, tr_acc[-1], te_acc[-1])
    
print(tr_acc)
print(te_acc)

In [ ]:
figure = plt.figure()
ax = plt.gca()

plt.semilogx(Ntrain * 5, tr_acc, c = "g", label = "Training")
plt.semilogx(Ntrain * 5, te_acc, c = "r", label = "Test")
ax.set_xlim(10, 40000)
ax.set_ylim(0.3, 0.9)

ax.set_xlabel('Training Size')
ax.set_ylabel('Accuracy')

plt.legend(frameon=False)

figure.savefig('training_size.pdf', bbox_inches='tight', pad_inches=0.1)

plt.show()

In [ ]:
X, y = blob_data(X = 5, Y = 5, max_K = 2, noise_level = 0.05, correlated = 0, N = 500)

mlp = neural_network.MLPClassifier(solver='lbfgs', alpha = 1e-2, random_state=1, hidden_layer_sizes=(100))
mlp.fit(X, y)

plot_classifier(X[:,], y[:], fn = mlp.predict, alpha = 0.4, alpha_pt = 0.01, file = "checkers_mlp.pdf")

In [ ]:
X, y = blob_data(X = 5, Y = 5, max_K = 2, noise_level = 0.05, correlated = 0, N = 500)

bdt = tree.DecisionTreeClassifier(min_samples_leaf = 15)
bdt.fit(X, y)

plot_classifier(X[:,], y[:], fn = bdt.predict, alpha = 0.4, alpha_pt = 0.01, file = "checkers_bdt.pdf")